# Chapter 10: Constraints

In [1]:
import numpy as np
import jax

## Algorithm 10.1

In [2]:
def penalty_method(f, p, x, min_f, k_max, rho=1, gamma=2):
    for k in range(k_max):
        x = min_f(lambda x_p: f(x_p) + rho*p(x_p), x)
        rho *= gamma
        if p(x) == 0:
            return x

    return x

### Example

In [3]:
def backtracking_line_search(f, gradient, x, d, alpha, p=0.5, beta=1e-4):
    y, g = f(x), gradient
    while f(x+alpha*d) > y + beta*alpha*(np.dot(g(x),d)):
        alpha *= p
    return alpha

def aproximate_line_search(f, x):
    '''
    One Step of Aproximate Line Search with Backtracking
    '''
    gradient = jax.grad(f)
    d = -gradient(x)

    alpha = backtracking_line_search(f, gradient, x, d, 1.0)

    return x + alpha*d

def f(x):
    return jax.numpy.sin(x[0]*x[1])+jax.numpy.exp(x[1]+x[2])-x[2]

p = lambda x: (x[0] + x[1] + x[2])**2
x = np.array([1.0, 2.0, 3.0])
k_max = 10
x_sol = penalty_method(f, p, x, aproximate_line_search, k_max)
print(x_sol)

[ 1.7996317  -1.419845   -0.37155974]


## Algorithm 10.2

In [ ]:
def augmented_lagrange_method(f, h, x, min_f, k_max, rho=1, gamma=2):
    
    lambda_ = np.zeros(len(h(x)))

    for k in range(k_max):
        p = lambda x: f(x) + rho/2*np.sum(h(x)**2) - np.dot(lambda_, h(x))
        x =  min_f(lambda x_p: f(x_p) + p(x_p), x)
        rho *= gamma
        lambda_ -= rho*h(x)

    return x

## Algorithm 10.3

In [ ]:
def interior_point_method(f, p, x, min_f, rho=1, gamma=2, epsilon = 0.001):
    delta = np.infty

    while delta > epsilon:
        x_prime = min_f(lambda x_p: f(x_p) + p(x_p)/rho, x)
        delta = (np.sum(x_prime - x)**2)**0.5
        x = x_prime
        rho *= gamma
    
    return x